# Exploring the CommaAI Device data

This requires python 3.11 or later because of the usage of the tools folder from Openpilot, which uses 3.11 features

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
import cv2
import pandas as pd
from pathlib import Path
# This is taken from the Openpilot project
from carla_experiments.tools.lib.logreader import LogReader

/tmp/ipykernel_2024524/1706577619.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


ModuleNotFoundError: No module named 'capnp'

In [ ]:
device_path = Path("/home/ulrikro/datasets/CommaAI/2024-01-14--13-01-26--10/")

# About the folder

Each folder, marked by their data and a number, contains 5 files: ecamera.hevc, fcamera.hevc, qcamera.ts, qlog, rlog. Each file contains 1 min video. 

* ecamera.hevc - road facing wide angle camera 1928*1208 at 20Hz
* fcamera.hevc - road facing narrow camera 1928*1208 at 20Hz
* qcamera.ts - driver facing wide angle camera 526*330 at 20Hz
* qlog - some
* rlog - some



In [ ]:
ecamera_path = device_path / "ecamera.hevc"
fcamera_path = device_path / "fcamera.hevc"
qcamera_path = device_path / "qcamera.ts"
qlog_path = device_path / "qlog"
rlog_path = device_path / "rlog"


# Overview of the video files

In [ ]:
def read_video_frames(file_path: str):
    frames = []
    video = cv2.VideoCapture(file_path)
    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break
        frames.append(frame)
    video.release()
    return frames

In [ ]:
ecamera_frames = read_video_frames(ecamera_path.as_posix())
len(ecamera_frames)

1200

In [ ]:
ecamera_frames[0].shape

(1208, 1928, 3)

In [ ]:
fcamera_frames = read_video_frames(fcamera_path.as_posix())
len(fcamera_frames)

1200

In [ ]:
fcamera_frames[0].shape

(1208, 1928, 3)

In [ ]:
qcamera_frames = read_video_frames(qcamera_path.as_posix())
len(qcamera_frames)

1200

In [ ]:
qcamera_frames[0].shape

(330, 526, 3)

# Overview of the log files

In [ ]:
qlog_data = np.load(qlog_path.as_posix())

ValueError: Cannot load file containing pickled data when allow_pickle=False